This project refers to an online retailer/wholesaler that sells through an e-commerce platform. The aim is to create a VIP club for important customers with access to extra perks. Customers will be selected according to the frequency and amount they buy. This program will be called “Insiders”. Using data manipulation techniques, customers will be grouped(clustered) according to similarities.




**Solution Planning (IOT)**

**Input**  

    - Business Problem: Select the most valuable customers to join a loyalty program.  
    - Dataset: E-commerce online sales.  
**Output**  

    1.List of customers who will join the Insiders program.  
    2.Answer questions  
    1.Who are the people eligible to join the Insiders program?  
    2.How many customers will be part of the group?  
    3.What are characteristics of these customers?  
    4.What is the percentage of revenue contribution coming from the Insiders?  
    5.What is the expected revenue of this group for the next months?  
**Tasks**  

1.Segment customers eligible to join the Insiders program. 
    •	What is being eligible? What are the most valuable customers?  
    •	Revenue:  
    
    - High average ticket  
    - High LTV (Life Time Value)  
    - Low Recency (last time of purchase)  
    - Big basket size  
    - Low churn probability  
   •   Cost:  
    - Low return rate  
2. How many customers will be part of the group?  
Total number of customers vs % of Insiders Group  
3. What are the main characteristics of these customers?  
•	Clustering attributes  
4. What is the percentage of revenue contribution coming from the Insiders?  
•	Anual total revenue  
•	Insiders' revenue  
5. What is the expected revenue of this group for the next months?  
•	Insiders' LTV
•	Cohort Analysis
•	Timeseries (ARIMA, ARMA, Prophet)
6. What actions can the marketing team take to increase revenue?  
•	Discount  
•	Shipping  
•	Company visit  
•	Perks  

**Benchmark guide: RFM Model:**

![RFM Model](./rfm.png)

The goal is to extract clusters that best fit the RFM(Recency, Frequency, Monetary) model.




## Imports

In [6]:
import os
import pandas  as pd
import numpy   as np
import seaborn as sns
import umap.umap_ as umap
import pickle
# import s3fs
import pvt.key

from s3fs.core import S3FileSystem

from matplotlib import pyplot as plt

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

from IPython.display import HTML
from IPython.core.display import Image

# from pandas_profiling import ProfileReport

from sklearn import cluster as c
from sklearn import metrics as m
from sklearn import preprocessing as pp
from plotly import express as px
from sklearn import decomposition as dd
from sklearn import ensemble as en
from sklearn import mixture as mx
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE

from scipy.cluster import hierarchy as hc

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

# home_path = '~/insider-clients-clustering/'
home_path = '../'


## Helper functions

In [7]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use('ggplot')
    plt.rcParams['figure.figsize'] = [24,9]
    plt.rcParams['font.size'] = 24
    
    display(HTML('<style>.container { width:100% !important;}</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()

jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## Load dataset

In [8]:


# df_raw = pd.read_csv(home_path + 'data/raw/data.csv', encoding = "ISO-8859-1")  # when exporting to linux machine, generates encoding error, not utf-8

# AWS bucket:
# os.environ['AWS_CONFIG_FILE'] = 'aws_config.ini'  # not needed for aws access
# os.environ['AWS_DEFAULT_REGION'] = 'eu-central-1'  # not needed for aws access


path_s3 = 's3://deploy-insiders-rdfaqz/insiders_db/data.csv'
os.environ['AWS_ACCESS_KEY_ID'] = pvt.key.AWS_ACCESS_KEY_ID
os.environ['AWS_SECRET_ACCESS_KEY'] = pvt.key.AWS_SECRET_ACCESS_KEY
s3 = S3FileSystem(anon=False)
df = pd.read_csv(s3.open(path_s3), encoding = "ISO-8859-1")




In [ ]:
df.shape

In [ ]:
df_raw.shape

In [ ]:
df_raw.head()

In [ ]:
df_raw.drop('Unnamed: 8',axis = 1, inplace = True)
df_raw.head()

# Data Preparation

In [ ]:
df1 = df_raw.copy()

## Columns Rename

In [ ]:
df1.columns

In [ ]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price','customer_id', 'country']

df1.columns = cols_new

## Data Dimensions

In [ ]:
print(f'Number of Rows:{df1.shape[0]}')
print(f'Number of Columns:{df1.shape[1]}')

## Data types

In [ ]:
df1.dtypes


## Check NA

In [ ]:
df1.isna().sum()

**135.080 customers id.... 'missing'**

## Filling NAs

Split dataset into missing vs not_missing:

In [ ]:
df_missing = df1.loc[df1['customer_id'].isna(),:]
df_not_missing = df1.loc[~df1['customer_id'].isna(),:]
df_missing.head()

In [ ]:
df_not_missing.shape

Dataset has 135.080 records where customer ID not present:

In [ ]:
df_missing.shape

In [ ]:
# missing_invoice = df_missing['invoice_no'].drop_duplicates().tolist()
# missing_invoice[0:10]

In [ ]:
# df_not_missing.loc[df_not_missing['invoice_no'].isin(missing_invoice), :].head()

- All customer_id from 19000 onwards will be a missing id just for training, and we will merge it on df1:

In [ ]:
# Decision:
# each NAN customer ID will be a new single customer time purchase, just for training(?)

# Create Reference:
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup),1) # linspace or arrange

# merging original w/ reference df:
df1 = pd.merge(df1, df_backup, on = 'invoice_no', how = 'left')

# need to keep columns...
# Coalesce:
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop aux columns
df1 = df1.drop(columns=['customer_id_x','customer_id_y'])
print(df1.isna().sum())
df1.head()



In [ ]:
# Removing NAs
df1 = df1.dropna(subset =['description','customer_id'])
print('Removed data: {:.5f} %, thats a total of {} rows.'.format(1 - (df1.shape[0] / df_raw.shape[0]), (df_raw.shape[0] - df1.shape[0] )))

In [ ]:
df1.isna().sum()

## Change dtypes

In [ ]:
df1.dtypes

In [ ]:
df1['customer_id'] = df1['customer_id'].astype(int)

In [ ]:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y' )
df1.head()

In [ ]:
df1.dtypes

## Descriptive Statistics

In [ ]:
# doing the model first, then come back - 'clustering not an end'.

Splitting dataset into numerical/categorical:

In [ ]:
num_attributes = df1.select_dtypes(include =['int64','float64', 'int32'])
cat_attributes = df1.select_dtypes(exclude =['int64','float64', 'int32'])

### Numerical Attributes

In [ ]:
# central tendency: mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T
# Dispersion - std dev, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

merged = pd.concat([ct1,ct2,d1,d2,d3,d4,d5,d6])
merged['index'] = ['Mean','Median', 'Std', 'Min','Max','Range','Skew','Kurtosis']
merged.set_index('index', inplace = True)
merged.T


1. Min for quantity is negative - returns?.
2. Unity Price 0.0 ? free item?


### Categorical Attributes

In [ ]:
#number of invoices with letters:
number_invs_letters = len(cat_attributes.loc[cat_attributes['invoice_no'].apply(lambda x: bool(re.search( '[^0-9]+', x))), 'invoice_no'].drop_duplicates())
print(f'Invoices with letters: {number_invs_letters}')

#number of stock codes with letters:
number_stk_letters = len(cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search( '[^0-9]+', x))), 'stock_code'].drop_duplicates())
print(f'Stock Codes with letters: {number_stk_letters}')

only_letters_unique_stock_code = cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x ) ) ), 'stock_code' ].unique()
print(f'Unique stock codes with only letters: {only_letters_unique_stock_code}')

In [ ]:
cat_attributes.sample(1)

In [ ]:
# some further investigation...
# df1.loc[df1['description'] == 'RED RETROSPOT PEG BAG'].sort_values('customer_id').reset_index()

# df1.loc[df1['invoice_no'] == 'C548995']

In [ ]:
df1.sample(1)

#### Invoice No

- Invoice number has letter numbers, and are mostly negative quantities.

In [ ]:
# There are invoices with letters
df_invoices_w_letters = df1.loc[df1['invoice_no'].apply(lambda x: bool (re.search('[^0-9]+', x))),:]

print('Total number of invoices w/ letters')
print(len(df_invoices_w_letters))

# Are all invoices that start with letters 'negative quantity' one?
print('Total number of invoices w/ letters with negative quantity')
print(len(df_invoices_w_letters[df_invoices_w_letters['quantity'] <0]))

# Yes, all invoices starting with letters are negative quantity except three / 'Adjust bad debt'.
print(df_invoices_w_letters[df_invoices_w_letters['quantity'] >= 0].head())
print(f"\nTotal # of non negative invoices: {df_invoices_w_letters[df_invoices_w_letters['quantity'] >= 0].invoice_no.count()} ")

#### Stock Code

In [ ]:
df1.loc[df1['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))),'stock_code'].unique()   

**To do:**  
Remove Stock_codes in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']  
New stock_Codes after joining missing data ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm',   'DCGSSBOY',        'DCGSSGIRL', 'PADS', 'B', 'CRUK']
 

#### Description

In [ ]:
# Todo - delete description

#### Country

In [ ]:
df1['country'].unique()

- UK main market:

In [ ]:
df1['country'].value_counts(normalize = True).head(6)

In [ ]:
# df1[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending = False).head()

# Data Filtering

In [ ]:
df2 = df1.copy()

Makes sense to filter some data: negative quantity values, assuming returns in the table.

2 ways - 
- exclude what has been returned, like customer had never bought the product?
- other way to create new feature based on returns. - Return feature would be beneficial to score clients?
(returns consumes resources)

Creating feature for returns.

In [ ]:
df2.loc[df2['unit_price'] > 0.0, ['customer_id', 'description', 'unit_price']].sort_values('unit_price', ascending = True).head(10)
# df1[['customer_id', 'description', 'unit_price']].sort_values('unit_price', ascending = True).head(10))


In [ ]:
df2.head()

- Filetering out: 
    - products with unit_price less than 4p(?), 
    - letters on stock_code, non-specific countries, 
    - and dropping 
    
- Auxiliary dataframes df2_returns and df2_purchase.

In [ ]:
# Numerical Attributes
# unit price > 0
df2 = df2.loc[df2['unit_price'] >= 0.04, :]  # there are products that price is 0.001

# Categorical Attributes
# stock code != [POST, D, M, DOT, CRUK]  
# New ones ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']
#df2 = df2[~df2['stock_code'].isin(['POST' 'D' 'M' 'PADS' 'DOT' 'CRUK'])]  # removing these products...
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]  # removing these products...

# Countries
# remove not specific like EU, and unspecified.
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

# Description - drop it
df2 = df2.drop(columns = 'description', axis = 1)

# quantity - assuming negative numbers are returns.
df2_returns = df2.loc[df2['quantity'] < 0,:]
df2_purchase = df2.loc[df2['quantity'] > 0,:]


# df2.loc[df2['quantity'] < 0,:].sort_values(['customer_id','description']).head()

# Feature Engineering

In [ ]:
df3 = df2.copy()
df3_purchase = df2_purchase.copy()
df3_returns = df2_returns.copy()

## Feature Creation

Here df3_ref will be the table compiled with features grouped by customers ID.

In [ ]:
# data referencing
df3_ref = df3.drop(['invoice_no','stock_code',
                   'quantity','invoice_date','unit_price',
                   'country'], axis = 1).drop_duplicates(ignore_index = True)
df3_ref.head()

# Gross Revenue ( qtt * price)
# Recency - latest purchase data for customer
# Frequency - how many products client bought in one year period

### Gross Revenue

Calculating the total ammount of revenue a customer has generated

In [ ]:
# Gross Revenue:
df3_purchase.loc[:,'gross_revenue'] = df3_purchase.loc[:,'quantity'] * df2_purchase.loc[:,'unit_price']

# # Monetary:
df_monetary = df3_purchase.loc[:,['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df3_ref = pd.merge(df3_ref, df_monetary, how = 'left', on = 'customer_id')
df3_ref.isna().sum()

### Recency - Days since last purchase

In [ ]:
# Recency - how long has it been since the last puchase.
df3_recency = df3_purchase[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df3_recency['recency_days'] = (df3_purchase['invoice_date'].max() - df3_recency['invoice_date']).dt.days  # using dt, to get int type
df3_recency = df3_recency[['customer_id','recency_days']].copy()

print('*******Nas before merge:')
print(df3_recency.isna().sum())

df3_ref = pd.merge(df3_ref, df3_recency, how = 'left', on = 'customer_id')

print('*******Nas After merge:')
print(df3_ref.isna().sum())
# 33 clients never bought anything, onlyu returns... likely data doesn´t comprehend these purchases.

### Ammount purchased

In [ ]:
# This it just the number different (variety of) products bought
df3_ammt_purchased = df3_purchase[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns ={'invoice_no':'invoice_ammt'})
df3_ref = pd.merge(df3_ref, df3_ammt_purchased, how = 'left', on = 'customer_id')
df3_ref.isna().sum()
df3_ref.head()

### Ammount products items  purchased

In [ ]:
# This it just the number of products bought
df3_i_ammt_purchased = df3_purchase[['customer_id', 'quantity']].groupby('customer_id').sum().rename(columns = {'quantity':'item_ammt'}).reset_index()
df3_ref = pd.merge(df3_ref, df3_i_ammt_purchased, how = 'left', on = 'customer_id')
df3_ref.isna().sum()
df3_ref.head()

### Ammount different products purchased

In [ ]:
# This it just the number of products bought
df3_ammt_type_of_prod_purchased = df3_purchase[['customer_id', 'stock_code']].groupby('customer_id').count().rename(columns = {'stock_code':'n_of_dif_products'}).reset_index()
df3_ref = pd.merge(df3_ref, df3_ammt_type_of_prod_purchased, how = 'left', on = 'customer_id')
print(df3_ref.isna().sum())
df3_ref.head()

### Average Ticket Value

In [ ]:
# Avg Ticket
df3_avg_ticket = df3_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue':'avg_ticket'})
df3_avg_ticket['avg_ticket'] = np.round(df3_avg_ticket['avg_ticket'],2 )
df3_ref = pd.merge(df3_ref, df3_avg_ticket, on = 'customer_id', how = 'left')
df3_ref.isna().sum()

# df_ref.head()

### Average Recency Days

In [ ]:
# df2.loc[df2['customer_id'] == 17850,:]

df3_aux = df3[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id','invoice_date'], ascending =['False','False'])
df3_aux['next_customer_id'] = df3_aux['customer_id'].shift() #next cx
df3_aux['previous_date'] = df3_aux['invoice_date'].shift() # next  invoice dt

df3_aux['avg_recency_days'] = df3_aux.apply(lambda x: (x['invoice_date'] - x['previous_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis = 1)

df3_aux = df3_aux.drop(['invoice_date', 'next_customer_id', 'previous_date'], axis =1).dropna()

#avg recency
df3_avg_recency_days = df3_aux.groupby('customer_id').mean().reset_index()

#Merging
df3_ref = pd.merge(df3_ref, df3_avg_recency_days, on ='customer_id', how = 'left' )

df3_ref.isna().sum()
# df_aux.dtypes


In [ ]:
df3_ref.head()

### Frequency

In [ ]:
### Frequency purchase

df3_max = df3[['customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').max()
df3_min = df3[['customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').min()
# df2_purchase = df2[['customer_id','invoice_date']].drop_duplicates().groupby('customer_id').count()
df3_purchase.head()

df3_aux = ( df2[['customer_id','invoice_no','invoice_date']].drop_duplicates()
                                                           .groupby('customer_id')
                                                           .agg( max_  = ('invoice_date','max'),
                                                                 min_  = ('invoice_date','min'),
                                                                 days_ = ('invoice_date',lambda x: ((x.max() - x.min() ).days) +1  ),
                                                                 buy_  = ('invoice_no', 'count' ) ) ).reset_index()

# frequency  # how many times a customer has bought given the specific period (number of days b/w
# 1st purchase and last purchase) eg 30 days, has bought 2 times = 2 / 30  = 1/15 = 0.066 ) 
# then... 'customer buys 0.06 a day' -maybe it should be # of times has bought last 12mo, and 
# if 1st purchase less than that, then use tome kind of formula to compensate for new customers


df3_aux['frequency'] = df3_aux[['buy_', 'days_']].apply(lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis = 1 )

# Merging
df3_ref = pd.merge(df3_ref, df3_aux[['customer_id', 'frequency']], on = 'customer_id', how = 'left')
# df_ref.head()
df3_ref.isna().sum()

# df_aux.sort_values('frequency', ascending = False).head(20)
# df_aux[df_aux['customer_id'] == 17850].head()

In [ ]:
df3_ref.head()

### Number of Returns

In [ ]:
df3_returns = df3_returns[['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'qtt_returns'})
df3_returns['qtt_returns'] = df3_returns['qtt_returns'] * -1

df3_ref = pd.merge(df3_ref, df3_returns, how = 'left', on ='customer_id')
df3_ref.loc[df3_ref['qtt_returns'].isna(), 'qtt_returns'] = 0
df3_ref.isna().sum()

### Basket Size - Number of product items per basket  *sum

In [ ]:
# invoice (basket) > products > number of total items for all products
# df2_purchase.head()

In [ ]:
# The total Quantity of products items divided by the number of times a customer has transactioned. 

df3_aux =(df3_purchase.loc[:,['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                      .agg(n_purchase = ('invoice_no', 'nunique'), 
                                                                           n_products = ('quantity','sum'))
                                                                      .reset_index() )


df3_aux['avg_basket_size'] = df3_aux['n_products'] / df3_aux['n_purchase']

df3_ref = pd.merge(df3_ref, df3_aux[['customer_id','avg_basket_size']], how = 'left', on = 'customer_id')
df3_ref.isna().sum()
# missing NA´s the ones in 'returns'

### Basket, distinctive items per purchase. *count

In [ ]:
# Transaction > product > item ammount
# The total number of different products divided by the number of times a customer transactioned.

df3_aux = (df3_purchase.loc[:,['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                       .agg(n_purchase = ('invoice_no', 'nunique'), 
                                                                            n_products = ('quantity','nunique'))
                                                                       .reset_index() )

df3_aux['avg_unique_basket_size'] = df3_aux['n_products'] / df3_aux['n_purchase']

df3_ref = pd.merge(df3_ref, df3_aux[['customer_id','avg_unique_basket_size']], how = 'left', on = 'customer_id')
df3_ref.isna().sum()

Check if returns will remain in df_ref:

In [ ]:
df3_ref.head()

In [ ]:
df3_ref.shape

# EDA

In [ ]:
df4 = df3_ref.dropna().copy()
# df4_debug = df3_ref.dropna().copy()  #delme
df4.isna().sum()

## Univariate Analysis

In [ ]:
# profile = ProfileReport(df4)
# profile.to_file('output.html')

### Gross Revenue

In [ ]:
sns.boxplot(df4.gross_revenue)

## Bivariate Analysis

**Low variance on Frequency and Avg. Ticket**

In [ ]:
# sns.pairplot(df4.drop(columns = 'customer_id'));

## Spatial Study

In [ ]:
# full_feat:
df43_full_features = df4.drop(columns = ['customer_id'], axis = 1).copy()

# embedded space:
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'n_of_dif_products', 'frequency', 'qtt_returns']
df43_less_features = df4[cols_selected].copy()
df43_less_features.drop(columns= 'customer_id', inplace = True)
# min-max, standard scales, robust scaler?

### Modeling with 2 sets 'full features', 'less features'

In [ ]:
# Many features were not useful, so trying modeling with 2 different sets


mm = pp.MinMaxScaler()

df43_full_features['gross_revenue'] = mm.fit_transform(df43_full_features[['gross_revenue']])  #*
pickle.dump(mm, open(home_path + 'features/gross_revenue_scaler.pkl', 'wb'))

# 'customer_id', 
df43_full_features['recency_days']          = mm.fit_transform( df43_full_features[['recency_days']])  # *
pickle.dump(mm, open(home_path + 'features/full_feat_recendy_days_scaler.pkl', 'wb'))
df43_full_features['invoice_ammt']          = mm.fit_transform( df43_full_features[['invoice_ammt']])
pickle.dump(mm, open(home_path + 'features/full_feat_invoice_ammt_scaler.pkl', 'wb'))
df43_full_features['item_ammt']             = mm.fit_transform( df43_full_features[['item_ammt']])  # *
pickle.dump(mm, open(home_path + 'features/full_feat_item_ammt_scaler.pkl', 'wb'))
df43_full_features['n_of_dif_products']     = mm.fit_transform( df43_full_features[['n_of_dif_products']])
pickle.dump(mm, open(home_path + 'features/full_feat_n_of_dif_products_scaler.pkl', 'wb'))
df43_full_features['avg_ticket']            = mm.fit_transform( df43_full_features[['avg_ticket']])
pickle.dump(mm, open(home_path + 'features/full_feat_avg_ticket_scaler.pkl', 'wb'))
df43_full_features['avg_recency_days']      = mm.fit_transform( df43_full_features[['avg_recency_days']])
pickle.dump(mm, open(home_path + 'features/full_feat_avg_recency_days_scaler.pkl', 'wb'))
df43_full_features['frequency']             = mm.fit_transform( df43_full_features[['frequency']])  #*
pickle.dump(mm, open(home_path + 'features/full_feat_frequency_scaler.pkl', 'wb'))
df43_full_features['qtt_returns']           = mm.fit_transform( df43_full_features[['qtt_returns']])  #*
pickle.dump(mm, open(home_path + 'features/full_feat_qtt_returns_scaler.pkl', 'wb'))
df43_full_features['avg_basket_size']       = mm.fit_transform( df43_full_features[['avg_basket_size']])
pickle.dump(mm, open(home_path + 'features/full_feat_avg_basket_size_scaler.pkl', 'wb'))
df43_full_features['avg_unique_basket_size']= mm.fit_transform( df43_full_features[['avg_unique_basket_size']])
pickle.dump(mm, open(home_path + '/features/full_feat_avg_unique_basket_size_scaler.pkl', 'wb'))


df43_less_features['gross_revenue'] = mm.fit_transform(df43_less_features[['gross_revenue']])
pickle.dump(mm, open(home_path + 'features/less_feat_gross_revenue_scaler.pkl', 'wb'))

df43_less_features['recency_days']          = mm.fit_transform( df43_less_features[['recency_days']])
pickle.dump(mm, open(home_path + 'features/less_feat_full_feat_recendy_days_scaler.pkl', 'wb'))

df43_less_features['n_of_dif_products']     = mm.fit_transform( df43_less_features[['n_of_dif_products']])
pickle.dump(mm, open(home_path + 'features/less_feat_full_feat_n_of_dif_products_scaler.pkl', 'wb'))

df43_less_features['frequency']             = mm.fit_transform( df43_less_features[['frequency']])
pickle.dump(mm, open(home_path + 'features/less_feat_frequency_scaler.pkl', 'wb'))

df43_less_features['qtt_returns']           = mm.fit_transform( df43_less_features[['qtt_returns']])
pickle.dump(mm, open(home_path + 'features/less_feat_qtt_returns_scaler.pkl', 'wb'))


df43_less_features.columns

In [ ]:
X_less_features = df43_less_features.copy()
X_full_features = df43_full_features.copy()

In [ ]:
df43_full_features.head()

In [ ]:
df43_less_features.head()

### PCA - full features

In [ ]:
X_full_features.head()

In [ ]:

pca = dd.PCA( n_components = X_full_features.shape[1])
principal_component = pca.fit_transform(X_full_features)

# plot explaining variables
features = range ( pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color = 'black')

# PCA COMPONENTS
df_pca = pd.DataFrame(principal_component)

In [ ]:
df_pca.head()

In [ ]:
sns.scatterplot(x = 0, y = 1, data = df_pca)
plt.title('PCA - full features[0 and 1]', fontsize = 18);

#### PCA - Less Features

In [ ]:
X_less_features.head()

In [ ]:
pca = dd.PCA( n_components = X_less_features.shape[1])
principal_components = pca.fit_transform(X_less_features)

# plot explaining variables
features = range ( pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color = 'black')

# PCA COMPONENTS
df_pca = pd.DataFrame(principal_components)

In [ ]:
sns.scatterplot( x=0, y=1, data=df_pca )

- Both PCAs show no clear spacing

### UMAP - full features

In [ ]:
# UMAP

# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(X_full_features)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df_pca )
plt.title('UMAP - full features', fontsize = 18);

#### UMAP - Less features

In [ ]:
# UMAP

# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(X_less_features)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df_pca )

### t-SNE

#### t-SNE - full features

In [ ]:
# UMAP
import umap.umap_ as umap
from sklearn.manifold import TSNE
# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = TSNE(n_components = 2, random_state = 42, n_jobs = -1)
embedding = reducer.fit_transform(X_full_features)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df_pca )
plt.title('t-SNE full features', fontsize = 18);

#### t-SNE - less features

In [ ]:
# UMAP
import umap.umap_ as umap
from sklearn.manifold import TSNE
# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = TSNE(n_components = 2, random_state = 42, n_jobs = -1)
embedding = reducer.fit_transform(X_less_features)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df_pca )

### Tree-Based Embedding - v1.0

In [ ]:
from sklearn import ensemble as en

In [ ]:
df4.head()
# X = df4.drop( columns = ['customer_id', 'gross_revenue'], axis = 1 )  # df4 - no transformation applied!!
# y = df4['gross_revenue'] # using gross_revenue

X_rf_full = X_full_features.drop(columns = ['gross_revenue'])
y_rf_full = X_full_features['gross_revenue']

# model definition
rf_model_full = en.RandomForestRegressor( random_state = 42, n_estimators = 100)

# model training
rf_model_full.fit( X_rf_full, y_rf_full)

#leaf
# rf_model.apply( X )  # applying own training data

#dataframe Leag

In [ ]:
df_leaf_full = pd.DataFrame(rf_model_full.apply(X_rf_full))
df_leaf_full.head()

In [ ]:
# Reducing dimensionality  # reduce the projection of the total columns columns into 2.
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf_full)

#embedding
df_tree_full = pd.DataFrame()
df_tree_full['embedding_x'] = embedding[:,0]
df_tree_full['embedding_y'] = embedding[:,1]

#plotting
sns.scatterplot(x = 'embedding_x',
               y = 'embedding_y',
               data = df_tree_full)


#### Tree-Based Embedding - v2.0

In [ ]:

X_less = X_less_features.drop(columns =['gross_revenue'], axis = 1)
y_less = X_less_features['gross_revenue']

# model definition
rf_model_less = en.RandomForestRegressor( random_state = 42, n_estimators = 100)

# model training
rf_model_less.fit( X_less, y_less)

pickle.dump(rf_model_less, open(home_path + 'models/rf_model_2.0.pkl', 'wb'))

#leaf
# rf_model.apply( X )  # applying own training data

#dataframe Leag

df_leaf_less = pd.DataFrame(rf_model_less.apply(X_less))
df_leaf_less.head()

In [ ]:
# Reducing dimensionality  # reduce the projection of the 100 columns into 2.
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf_less)

#embedding
df_tree_v2 = pd.DataFrame()
df_tree_v2['embedding_x'] = embedding[:,0]
df_tree_v2['embedding_y'] = embedding[:,1]

#plotting
sns.scatterplot(x = 'embedding_x',
               y = 'embedding_y',
               data = df_tree_v2)

#### Tree based embedded - simple - in use

In [ ]:
X = df4.drop( columns = ['customer_id', 'gross_revenue'], axis = 1 )  # df4 - no transformation applied!!
y = df4['gross_revenue'] # using gross_revenue

# model definition
rf_model = en.RandomForestRegressor( random_state = 42, n_estimators = 100)

# model training
rf_model.fit( X, y)



df_leaf = pd.DataFrame(rf_model. apply(X))
df_leaf.head()



In [ ]:
# Reducing dimensionality  # reduce the projection of the total columns columns into 2.
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf)

#embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]

#plotting
sns.scatterplot(x = 'embedding_x',
               y = 'embedding_y',
               data = df_tree)
plt.title('Forest Tree Regressor embedding followed by UMAP reduction', fontsize = 18);

# Data Preparation


In [ ]:
# df5 = df4.copy()
df5_aux = df4.copy()
df5_df_tree = df_tree.copy()
df5_df_tree_v2 = df_tree_v2.copy()

In [ ]:

sns.distplot(df5_aux['gross_revenue']);
# Talvez um minmax, tá muito deseguilibrada essa disribuicao.


In [ ]:
sns.distplot(np.log(df5_aux['gross_revenue']))

In [ ]:
# Previously tested transformations before deciding going with embedded spaces:


# # mm = pp.MinMaxScaler()
# # ss = pp.StandardScaler()
# # rs = pp.RobustScaler()

# df5_aux['gross_revenue'] = mm.fit_transform(df5_aux[['gross_revenue']])
# df5_aux['recency_days'] = mm.fit_transform(df5_aux[['recency_days']])
# df5_aux['invoice_ammt'] = mm.fit_transform(df5_aux[['invoice_ammt']])
# df5_aux['item_ammt'] = mm.fit_transform(df5_aux[['item_ammt']])
# df5_aux['n_of_dif_products'] = mm.fit_transform(df5_aux[['n_of_dif_products']])

# # df5_aux['avg_ticket'] = mm.fit_transform(df5_aux[['avg_ticket']])
# # df5_aux['avg_recency_days'] = mm.fit_transform(df5_aux[['avg_recency_days']])

# df5_aux['frequency'] = mm.fit_transform(df5_aux[['frequency']])
# df5_aux['qtt_returns'] = mm.fit_transform(df5_aux[['qtt_returns']])

# # df5_aux['avg_basket_size'] = mm.fit_transform(df5_aux[['avg_basket_size']])
# # df5_aux['avg_unique_basket_size'] = mm.fit_transform(df5_aux[['avg_unique_basket_size']])  

# # avg wont make sense in the end, not looking for avg of the avg in the end.

# variable = 'gross_revenue'

# # ['customer_id', 'gross_revenue', 'recency_days', 'n_of_dif_products', 'frequency', 'qtt_returns']

# Data as it is
# print('Min:{} - Max:{}'.format(df5_aux[variable].min(), df5_aux[variable].max() ) )
# sns.displot(df5_aux[variable]);

# # Data Standardized/Rescaled
# print('Min:{} - Max:{}'.format(df5_aux[variable].min(), df5_aux[variable].max() ) )
# sns.displot(df5_aux[variable]);

In [ ]:
# Box Plot
# sns.boxplot(df5_aux[variable]);

# Feature Selection

In [ ]:
# Previous cycles:
# cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'n_of_dif_products', 'frequency', 'qtt_returns']

In [ ]:
# df6 = df5[cols_selected].copy()
df6_df_tree = df5_df_tree.copy()
df6_df_tree_v2 = df5_df_tree_v2.copy()

df6_df_tree.head()


# Hyperparameter Fine-Tuning

In [ ]:
# X = df6.drop(columns = ['customer_id'])
X = df6_df_tree.copy()
X_v2 = df6_df_tree_v2.copy()

In [ ]:
X.head()

In [ ]:
# clusters = [2, 3, 4, 5, 6, 7 , 8 ,9 ]
cluster_hyperpam_testing = np.arange(2,25,1)

## K-Means

In [ ]:

kmeans_list = []
for k in cluster_hyperpam_testing:
    # defining model
    kmeans_model = c.KMeans(n_clusters = k)

    # defining training
    kmeans_model.fit(X)

    # define prediction
    labels_71_kmeans_v1 = kmeans_model.predict(X)
    # performance
    # Using silhouette inside metrics package.
    kmeans_list.append( m.silhouette_score(X, labels_71_kmeans_v1, metric = 'euclidean') )
    

#### v2.0 

kmeans_list_v2 = []
for k in cluster_hyperpam_testing:
    # defining model
    kmeans_model_v2 = c.KMeans(n_clusters = k)

    # defining training
    kmeans_model_v2.fit(X_v2)

    # define prediction
    labels_71_kmeans_v2 = kmeans_model_v2.predict(X_v2)
    # performance
    # Using silhouette inside metrics package.
    kmeans_list_v2.append( m.silhouette_score(X_v2, labels_71_kmeans_v2, metric = 'euclidean') )

In [ ]:
plt.plot(cluster_hyperpam_testing, kmeans_list, linestyle='--', marker = 'o', color ='b')
plt.plot(cluster_hyperpam_testing, kmeans_list_v2, linestyle='--', marker = 'o', color ='green')
plt.xlabel('K')
plt.ylabel(' Silhouette Score ')
plt.title('KMeans - Silhouette Score x K - Performance Comparison', fontsize = 16)



blue_patch = mpatches.Patch(color='blue', label='Raw data - simple')
green_patch = mpatches.Patch(color='green', label='MinMax Transformation')
yellow_patch = mpatches.Patch(color='yellow', label='MinMax Transformation')

plt.legend(handles=[blue_patch, green_patch]);



## GMM

In [ ]:
# def, training, predict, score
gmm_list =[]
for k in cluster_hyperpam_testing:
    gmm_model = mx.GaussianMixture(n_components = k)

    gmm_model.fit(X)

    labels_72_GMM = gmm_model.predict(X)

    sil_gmm_v1 = m.silhouette_score(X, labels_72_GMM, metric = 'euclidean')
    gmm_list.append(sil_gmm_v1)

    
### v2.0
gmm_list_v2 =[]
for k in cluster_hyperpam_testing:
    gmm_model_v2 = mx.GaussianMixture(n_components = k)

    gmm_model_v2.fit(X_v2)

    labels_72_GMM_v2 = gmm_model_v2.predict(X_v2)

    sil_gmm_v2 = m.silhouette_score(X_v2, labels_72_GMM_v2, metric = 'euclidean')
    gmm_list_v2.append(sil_gmm_v2)
    

In [ ]:
plt.plot(cluster_hyperpam_testing, gmm_list, linestyle='--', marker = 'o', color = 'b')
plt.plot(cluster_hyperpam_testing, gmm_list_v2, linestyle='--', marker = 'o', color = 'green')
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.title('GMM Silhouette Score K');

## Hierarchical Clustering

In [ ]:
# model definition and training
hc_model = hc.linkage(X, 'ward')
hc_model_v2 = hc.linkage(X_v2, 'ward')



In [ ]:
# DENDROGRAM

# hc.dendrogram(
#     hc_model,
#     leaf_rotation = 90,
#     leaf_font_size = 8)
# plt.plot()

In [ ]:
hc.dendrogram(
    hc_model,
    truncate_mode='lastp',
    p=12,
    leaf_rotation = 90,
    leaf_font_size = 8,
    show_contracted = True)
plt.title('Dendogram simplified', fontsize = 18)
plt.show()


### H-Clustering Silhouette Score

In [ ]:
hc_list = []
for k in cluster_hyperpam_testing:
    # def, training, predict score
    hc_model = hc.linkage(X, 'ward')

    # prediction
    labels_hc_v1 = hc.fcluster(hc_model, k, criterion = 'maxclust')

    #metrics
    sil_hc_v1 = m.silhouette_score(X, labels_hc_v1, metric = 'euclidean')
    hc_list.append( sil_hc_v1 )

    
#### v2.0
hc_list_v2 = []
for k in cluster_hyperpam_testing:
    # def, training, predict score
    hc_model_v2 = hc.linkage(X_v2, 'ward')

    # prediction
    labels_hc_v2 = hc.fcluster(hc_model_v2, k, criterion = 'maxclust')

    #metrics
    sil_hc_v2 = m.silhouette_score(X_v2, labels_hc_v2, metric = 'euclidean')
    hc_list_v2.append( sil_hc_v2 )

In [ ]:
plt.plot(cluster_hyperpam_testing, hc_list, linestyle = '--', marker ='o', color = 'b')
plt.plot(cluster_hyperpam_testing, hc_list_v2, linestyle = '--', marker ='o', color = 'green')
plt.xlabel('k')
plt.ylabel('')

blue_patch731 = mpatches.Patch(color='b', label='v1 - more features')
green_patch731 = mpatches.Patch(color='green', label='v2 - lesser features')

plt.legend(handles=[blue_patch731, green_patch731])
plt.title('K performance and Silhouette Score', fontsize = 18)

### H-Clustering Silhoutte plots

- Even though Hierarchical Clustering points out optimum k at a higher value, that becomes impractical in terms of RFM modeling, so checking performance between 8 and 14 groups.

In [ ]:
h7_clusters_interest = [8,9,10,11,12,13,14]
# h7_clusters_interest = [8] #,9,10,11,12,13,14]

In [ ]:
fig, ax = plt.subplots(4,2)
fig.set_size_inches(25,20)  #setting a size for fig

print ('HC starting counting clusters at 1, not 0.')
for k in h7_clusters_interest:
    q,mod = divmod(k  - h7_clusters_interest[0],2)
    ax[q, mod].set_xlim([ -0.1, 1])
    ax[q, mod].set_ylim([ 0, len(X) + (k+1)*10])
    #model def
    hc_model = hc.linkage(X, 'ward')

    # predict
    labels = hc.fcluster(hc_model, k, criterion = 'maxclust')
    
    # print(f'Labels given by current model {np.unique(labels)}')
    # performance
    ss = m.silhouette_score(X, labels, metric = 'euclidean')
    silhouette_avg = ss
    print('When k {}, Silhouette Score: {}'.format(k, ss))
    samples_silhouette_values = m.silhouette_samples(X, labels)
    y_lower = 10
    
    
    for i in range(1, k +1):
        # select clusters
        ith_samples_silhouette_values = samples_silhouette_values[labels == i]

        #sorting
        ith_samples_silhouette_values.sort()
        
        # size_cluster
        size_cluster_i = ith_samples_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        cmap = cm.get_cmap('Spectral')
        color = cmap(i/k)
        # fill_betweenx  (matplotlib)
        ax[q,mod].fill_betweenx( 
            np.arange(y_lower, y_upper), 0, 
            ith_samples_silhouette_values,
            )
        
        
        ax[q, mod].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10


    ax[q, mod].set_ylim([0, len(X) + (k + 1) * 10])
    ax[q, mod].set_title(f'k {k}')
    ax[q, mod].set_xlabel(f'Silhouette score')
    ax[q, mod].set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    
    ax[q, mod].set_yticks([])
    ax[q, mod].set_ylabel('Clusters')

    ax[q, mod].axvline(x=silhouette_avg, color="red", linestyle="--")
    
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

## DBSCAN

In [ ]:
eps = 1.2
min_samples = 33
dbscan_model = c.DBSCAN( eps = eps, min_samples = min_samples) 

labels_db_scan = dbscan_model.fit_predict(X)

db_scan_sil = m.silhouette_score( X, labels_db_scan, metric = 'euclidean')

print('Silhouette Score {} '.format(db_scan_sil))
print('Clusters k includes noisy one (-1) if present: {}'.format(len(unique(labels_db_scan) )))
print('labels {} '.format(unique(labels_db_scan)))
db_scan_k_v1 = len(unique(labels_db_scan)) - 1 # we will need that variable in the results

print('\n\nVersion2:')
dbscan_model_v2 = c.DBSCAN( eps = eps, min_samples = min_samples) 

labels_db_scan_v2 = dbscan_model_v2.fit_predict(X_v2)

db_scan_sil_v2 = m.silhouette_score( X_v2, labels_db_scan, metric = 'euclidean')

print('Silhouette Score {} '.format(db_scan_sil_v2))
print('Clusters k includes noisy one (-1) if present: {}'.format(len(unique(labels_db_scan_v2) )))
print('labels {} '.format(unique(labels_db_scan_v2)))



### Neighbors issue - tuning

In [ ]:
# checking on v1
neighbors = NearestNeighbors(n_neighbors = min_samples).fit(X)
distances, indices = neighbors.kneighbors(X)

In [ ]:
distances = np.sort(distances, axis = 0)
distances = distances[:,1]
plt.plot(distances)

In [ ]:
# zooming in
plt.plot(distances[2900:])

## RESULTS

In [ ]:
db_scan_sil

In [ ]:
# db_scan_k

In [ ]:
df_results = pd.DataFrame(
    {'KMeans':kmeans_list, 'GMM': gmm_list,
     'HCList':hc_list}).T

dbscan_list = []
for a in (df_results.columns):
    if a != db_scan_k_v1:
        dbscan_list.append(0.0)
    else:
        dbscan_list.append(db_scan_sil)

print(dbscan_list)
    
df_results = pd.DataFrame(
    {'KMeans':kmeans_list, 'GMM': gmm_list,
     'KMeans_v2':kmeans_list_v2, 'GMM_v2': gmm_list_v2,
     'HCList_v2':hc_list_v2,'HCList':hc_list, 'DBScan':dbscan_list}).T

df_results.columns = cluster_hyperpam_testing
# df_results = df_results.iloc[:,7:]
df_results.style.highlight_max(color = 'lightgreen', axis = 1)

- General optimum for k here would be range from 14 to 24. However for business team that would represent a lot of groups to be working with.

## Within-Cluster Sum of Square (WSS)

In [ ]:
# trying out diffente cluster values

# clusters = [2, 3, 4, 5, 6, 7 ]
h77_clusters = np.arange(2,14,1)

In [ ]:
wss = []
for k in h77_clusters:
    # model definition
    kmeans = c.KMeans( init = 'random', n_clusters = k, n_init = 10, max_iter = 300, random_state = 42)  # doing centroids at random # n_init, centroid starting times(?)
    # model training
    kmeans.fit(X)
    # validation
    wss.append(kmeans.inertia_)

# plootting wss - ELBOW method # We want to findout the best k before going for the final model
# like a pre-training model.
# This graph is supposed to look like an elbow... (?)
# Supposed to choose the point in the graph where the changes(The angle) most significative happen in this case
# happens where K = 3 and 5 - it is up to the Data Scientist to choose the best point.b

plt.plot(h77_clusters, wss, linestyle = '--', marker ='o', color='b')
plt.xlabel('K')
plt.ylabel('Within-Cluster Sum of Square');
plt.title('WSS vs K')

In [ ]:
# Searching for K, using the library yellowbrick
from yellowbrick.cluster import KElbowVisualizer
# !pip install yellowbrick.cluster

kmeans = KElbowVisualizer(c.KMeans(), k=h77_clusters, timing = False)
kmeans.fit(X)
kmeans.show()  # getting best value k 

# knee method....

## Sillhouette Score

In [ ]:
kmeans_si = KElbowVisualizer(c.KMeans(), k=h77_clusters, metric = 'silhouette', timings = False)
kmeans_si.fit(X)
kmeans_si.show()

## Silhouette Analysis - KMeans

In [ ]:
h79_clusters = [8,9,10,11,12,13]

fig, ax = plt.subplots(3,2, figsize = (25,18))
pos = 1
posi = 1
for k in h79_clusters:
    km = c.KMeans(n_clusters = k, init = 'random', n_init = 10, max_iter = 100, random_state = 42)
    q, mod = divmod(k, 2)

    visualizer = SilhouetteVisualizer(km, colors = 'yellowbrick', ax=ax[posi - 1][mod])
    
    if pos % 2 == 0:
        posi += 1
    pos += 1
    
    visualizer.fit(X)
    visualizer.finalize()

Silhouette now giving us much insight on the number of clusters. A lot more organized after opting for the embedded tree space.

# Model Training

## K-Means

In [ ]:
# model definition
# Opting for 11 - it is not the optimum division, however 
# in terms of business and RFM goal, makes sense to go with a lower number,
# silhouette score not the optimum, ok given the kind of problem.

k = 11
kmeans = c.KMeans(init = 'random', n_clusters = k, n_init = 10, max_iter = 300, random_state = 42)

# model training
kmeans.fit(X)

# clustering
# labels = kmeans.predict(X)
labels = kmeans.labels_


## Cluster Validation

In [ ]:
# from sklearn import metrics as m

In [ ]:
# WSS (Whithin Cluster Sum of Squares)
print( f'WSS Value: {kmeans.inertia_}')

# SS ( Silhouette Score)
print(f'SS Value: {m.silhouette_score(X, labels, metric = "euclidean")}')

# Cluster Analysis

- At this moment we are not applying the better model(hc) for this separation - todo: next review cycle.

## Visual Inspection

In [ ]:
# df9 = df6.copy()
df9 = X.copy()
df9['cluster'] = labels
df9.head()

In [ ]:
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df9, hue = 'cluster', palette = 'deep')
plt.title('Visual Cluster Representation - k = 11, Forest Tree Regressor followed by UMAP > KMeans ', fontsize = 18)

### Multi-Feature

In [ ]:
visualizer = SilhouetteVisualizer(kmeans, colors = 'yellowbrick')
visualizer.fit(X)
visualizer.finalize()

### 2d plot

In [ ]:
df9.columns

In [ ]:
# previous cycle left over:
# df_viz = df9.drop(columns = 'customer_id', axis = 1)
# sns.pairplot(df_viz, hue = 'cluster')

### UMAP - t-SNE

In [ ]:
# Manifold
# UMAP, t-SNE(2009) - MAnifold - Topology


In [ ]:
# previou cycle left over:

# # UMAP
# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
# reducer = umap.UMAP(random_state = 42)
# embedding = reducer.fit_transform(X)

# # embedding
# df_viz['embedding_x'] = embedding[:,0]
# df_viz['embedding_y'] = embedding[:,1]

# #plotting umap
# sns.scatterplot( x = 'embedding_x', y = 'embedding_y', 
#                 hue = 'cluster', 
#                 palette = sns.color_palette('hls', n_colors = len( df_viz['cluster'].unique() ) ), data = df_viz )

## Cluster Profile


In [ ]:
df9.head()

In [ ]:
df4.gross_revenue.sum()

In [ ]:

df9_aux = df4.copy()
df9_aux['cluster'] = labels

# Number of Customers
df_cluster = (df9_aux[['customer_id', 'cluster']].groupby('cluster').count().reset_index())
df_cluster['perc_customer'] = 100 * (df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# Centroids - the group mean. 

# Avg Gross Revenue
df_avg_gross_revenue = (df9_aux[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, on = 'cluster', how = 'left')

# Avg recency days
df_avg_recency_days = (df9_aux[['recency_days', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_avg_recency_days, on = 'cluster', how = 'left')

# Avg invoices number
df_inv = (df9_aux[['n_of_dif_products', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_inv, on = 'cluster', how = 'left')

# Frequency
df_fre = (df9_aux[['frequency', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_fre, on = 'cluster', how = 'left')

# Returns
df_ret = (df9_aux[['qtt_returns', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_ret, on = 'cluster', how = 'left')

df_cluster


In [ ]:
df_cluster['frequency_min_max'] = mm.fit_transform(df_cluster[['frequency']])
df_cluster['recency_min_max'] = mm.fit_transform(df_cluster[['recency_days']])
df_cluster['recency_min_max'] = 1 - df_cluster['recency_min_max']
df_cluster['bubble'] = df_cluster.gross_revenue // 10
df_cluster.loc[3, 'bubble'] = 1400.0

In [ ]:
df_cluster.sort_values('gross_revenue', ascending = False)
df_cluster['gross_revenue_log'] = np.log10(df_cluster['gross_revenue'])

In [ ]:
sns.scatterplot( x = 'recency_min_max', y = 'frequency_min_max', data = df_cluster, hue = 'cluster', palette = 'deep', size = 'gross_revenue_log', sizes = (100,1500) )
plt.title('Clusters organized according to frequency and recency - minmax', fontsize = 16)
plt.figtext(0.5,0.02, 'The size of the circles represent the monetary gross revenue - this graph\nis a simple abstraction from the RFM(Recency, Frequency, Monetary model.)', horizontalalignment =  'center');

In [ ]:
cluster_names = {
    3 : 'Champion - Insiders',
    7 : 'Loyal Customers',
    5 : 'Promising recent 1',
    4 : 'Promising recent 2',
    6 : 'Promising 3',
    9 : 'Attention needed - about to sleep',
    0 : 'Attention needed 1',
    10 :'Attention needed 2',
    1 : 'Hybernating 1',
    2 : 'Hybernating 2',
    8 : 'About to lose'}
df_cluster['cluster_name'] = df_cluster['cluster'].map(cluster_names)

In [ ]:
df_cluster

### Cluster Synthesis

In [ ]:
# df_cluster.columns

In [ ]:
df_cluster.columns = ['cluster', 'number_of_customers', 'perc_customer', 'gross_revenue',
       'recency_days', 'n_of_dif_products', 'frequency', 'qtt_returns',
       'frequency_min_max', 'recency_min_max', 'aux_bubble', 'gross_revenue_log','cluster_name'
       ]

In [ ]:
df_cluster_rep = df_cluster.sort_values('cluster').reset_index(drop = True)

for i in range(len(np.unique(df_cluster_rep['cluster']))):
    print('Cluster {}: "{}"'.format(df_cluster_rep.loc[i, 'cluster'], df_cluster_rep.loc[i, 'cluster_name']))
    print(' - Number of customers {} ({:,.2f} % of total)'.format(df_cluster_rep.loc[i, 'number_of_customers'], df_cluster_rep.loc[i, 'perc_customer']) )
    print(' - Average amount spent in total ${:,.2f} '.format(df_cluster_rep.loc[i, 'gross_revenue']) )
    print(' - Average Recency (days) {:,.2f} \n'.format(df_cluster_rep.loc[i, 'recency_days']) )

## Hypothesis Testing

### MindMap

In [ ]:
df10 = df9_aux.copy()
df10['cluster_name'] = df10['cluster'].map(cluster_names)
# df10.sample(8)

- Modeling the phenomena.
- Understanding entities (customer, location, family, income)
- Entity properties (eg. customer: income, education, age group, etc)


In [ ]:
from IPython.core.display import Image
Image(home_path + 'mindmap.png')

### Hypothesis Purchases: 

#### Insiders cluster customers represent more than 20% of total revenue.

In [ ]:
insiders = df10.loc[df10['cluster'] == 3, 'gross_revenue'].sum()
# df_sales_not_insiders = df10.loc[df10['cluster'] == 3, 'item_ammt'].median()
total =df10.loc[:, 'gross_revenue'].sum()
print ( 'insiders / total: {:.2f}%'.format(insiders / total * 100) )

TRUE. Insiders represent more than 30% of gross revenue

####  Customers in the Insiders cluster have a number of returns below the average of the total customer base.


In [ ]:
# avg returns inside
df_avg_return_insiders = df10.loc[df10['cluster'] == 3, 'qtt_returns'].mean()

# avg return total
df_avg_return_all = df10['qtt_returns'].mean()

#
print( 'Avg Return Insiders: {} vs Avg Return All:{}'.format( np.round( df_avg_return_insiders, 0 ), 
                                                              np.round( df_avg_return_all, 0 ) ) )

FALSE. Average return for insiders are higher.

#### Cheaper products are more transactioned than expensive ones.

In [ ]:


bins = list(range(0,1100,100))
df2['binned_price'] = pd.cut(df2['unit_price'], bins=bins)

aux2 = df2[['unit_price','binned_price']].groupby('binned_price').count().reset_index()
# plt.subplot(1,3,2)
# sns.barplot(x = 'binned_price', y = 'unit_price', data = aux2)
aux2.columns = ['binned_price', 'transactions_ammt']
aux2

TRUE. Most of transactions comprehend products up to 100 euros.

#### Customers in the Insiders program account for more than 30% of company gross revenue.

In [ ]:
# defining dataset with cluster names
# names = {"4" : "Insiders", "1" : "Potential Loyalists", "0" : "Loyal Customers", "2" : "Promissing 1", "8" : "Need Attention 1", "6" : "Promising 2", "7" : "Need Attention 2", "5" : "Hibernating 1", "10" : "Churn 1", "3" : "Price Sensitive", "9" : "Churn 2", "11" : "Hibernating 2", "-1" : 'Noise' }


# summing revenue by cluster
df_agg = df10[['cluster', 'gross_revenue']].copy()
df_agg = df_agg.groupby('cluster').sum().reset_index()
df_agg['cluster_name'] = df_agg['cluster'].map(cluster_names)
# df_agg

In [ ]:
# ploting the results
bar = sns.barplot(data = df_agg, x = 'cluster_name', y = 'gross_revenue', palette = 'husl'); 
plt.title("Gross Revenue and % of customers in each group", fontsize = 22);
#plt.figtext(0.5,0, 'he')
bar_order = list(df_agg['cluster_name'].unique())
spots = zip(bar.patches, bar_order)
for spot in spots:
    total = df_agg['gross_revenue'].sum()
    class_total = df_agg[df_agg['cluster_name'] == spot[1]]['gross_revenue']
    
    percent = float(class_total/total*100)

    height = spot[0].get_height()
    bar.text(spot[0].get_x()+0.3, height+5, '{:.2f}%'.format(percent), fontsize = 14 )


TRUE. They account for almost 31,79%.

### Business questions

#### Who are the people elegible for the Insiders program?

In [ ]:
insiders_id = list(df10.loc[df10['cluster'] == 3, 'customer_id'])
print(insiders_id)

#### How many people are part of this group?

In [ ]:
print(df10.loc[df10['cluster'] == 3, 'customer_id'].size)

#### What are the main characteristics of the insiders group?

In [ ]:
print(f"Total Number of Insiders: {len(df10.loc[df10['cluster'] == 3, 'customer_id'])} out of {len(df10['customer_id'])}")
print(f"Average Gross Revenue Insiders: {df10.loc[df10['cluster'] == 3, 'gross_revenue'].mean():.2f}, Overall: {df10['gross_revenue'].mean():.2f}")
print(f"Median Gross Revenue Insiders: {df10.loc[df10['cluster'] == 3, 'gross_revenue'].median():.2f}, Overall: {df10['gross_revenue'].median():.2f}")
print(f"Average Recency Insiders: {df10.loc[df10['cluster'] == 3, 'recency_days'].mean():.2f}, Overall: {df10['recency_days'].mean():.2f}")
print(f"Average Frequency Insiders: {df10.loc[df10['cluster'] == 3, 'frequency'].mean():.2f}, Overall: {df10['frequency'].mean():.2f}")
print(f"Average different products purchased Insiders: {df10.loc[df10['cluster'] == 3, 'avg_basket_size'].mean():.2f}, Overall: {df10['avg_basket_size'].mean():.2f}")


In [ ]:
# Break here

# Deploy to production

In [ ]:

# from sqlalchemy import create_engine

# host = 
# port = 5432
# database ='postgres'
# user = 'postgres'
# pwd = 
# endpoint = f'postgresql://{user}:{pwd}@{host}:{port}'

# conn = create_engine(endpoint)



In [ ]:
# query_table_df10_creation = '''
#     CREATE TABLE df_10_insiders (
#         customer_id            INTEGER,
#         gross_revenue          REAL,
#         recency_days           INTEGER,
#         invoice_ammt           INTEGER,
#         item_ammt              INTEGER,
#         n_of_dif_products      INTEGER,
#         avg_ticket             REAL,
#         avg_recency_days       REAL,
#         frequency              REAL,
#         qtt_returns            INTEGER,
#         avg_basket_size        REAL,
#         avg_unique_basket_size REAL,
#         cluster                INTEGER,
#         cluster_name           TEXT
#     )
#     '''

# query_table_df_cluster_creation = '''
#     CREATE TABLE df_cluster (
#         cluster                INTEGER,
#         number_of_customers    INTEGER,
#         perc_customer          REAL,
#         gross_revenue          REAL,
#         recency_days           REAL,
#         n_of_dif_products      REAL,
#         frequency              REAL,
#         qtt_returns            REAL,
#         frequency_min_max      REAL,
#         recency_min_max        REAL,
#         aux_bubble             REAL,
#         gross_revenue_log      REAL,
#         cluster_name           TEXT
#     )
#     '''
        

# conn.execute(query_table_df10_creation)
# conn.execute(query_table_df_cluster_creation)


In [ ]:
# df10.to_sql( 'df_10_insiders', con=conn, if_exists='append', index=False )

# df_cluster.to_sql( 'df_cluster', con=conn, if_exists='append', index=False )

In [ ]:
# conn.close()